In [16]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Kitti_Yolo_Tracking

# Install python libs

In [ ]:
!pip install ultralytics opencv-python matplotlib onnxruntime onnxruntime-tools --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 78.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 98.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.7/212.7 kB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.5/17.5 MB 99.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 9.9 MB/s eta 0:00:00


# Get KITTI Dataset

In [ ]:
!python download_kittidataset.py

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Extracting data_object_image_2.zip...
Extracting data_object_label_2.zip...
KITTI dataset ready!


# Convert the Ground-truth label files ( KITTI -> YOLO) & Split dataset into train & val

In [ ]:
!python label_convertor.py "kitti2yolo"

KITTI -->  YOLO conversion complete!
Train images: 5984
Val images:   1497


# Trigger model training

In [ ]:
from ultralytics import YOLO

model = YOLO("yolov8n.pt")  # or yolov8s.pt
model.train(
        data="kitti.yaml",
        epochs=50,
        imgsz=640,
        batch=16,
    )

Ultralytics 8.4.14 🚀 Python-3.12.12 torch-2.9.0+cpu CPU (Intel Xeon CPU @ 2.20GHz)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=kitti.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, end2end=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=0.0, name=train2, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, p

KeyboardInterrupt: 

# Generate mAP50 & mAP50-95 Metrics

In [ ]:
!python evaluate_yolo.py

Ultralytics 8.4.14 🚀 Python-3.12.12 torch-2.9.0+cpu CPU (Intel Xeon CPU @ 2.20GHz)
Model summary (fused): 73 layers, 3,006,233 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access ✅ (ping: 2.3±3.8 ms, read: 47.4±6.3 MB/s, size: 804.6 KB)
val: Scanning /content/drive/MyDrive/kitti_yolo_project/Kitti_Yolo_Tracking/kitti_yolo/labels/val.cache... 1497 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 1497/1497 139.5Mit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 94/94 2.1s/it 3:17
                   all       1497       6882      0.883      0.784      0.865      0.596
                   Car       1333       5680      0.928      0.906       0.96       0.78
            Pedestrian        357        896       0.88      0.671      0.795      0.441
               Cyclist        222        306      0.841      0.777      0.839      0.567
Speed: 1.4ms preprocess, 89.5ms inference, 0.0ms loss, 0.4ms postprocess per image


# Visualize Ground-truth label vs Predictions for the particular image 

In [ ]:
!python visualize_predictions.py


image 1/1 /content/drive/MyDrive/kitti_yolo_project/Kitti_Yolo_Tracking/kitti_yolo/images/val/000003.png: 224x640 2 Cars, 169.4ms
Speed: 7.6ms preprocess, 169.4ms inference, 1.7ms postprocess per image at shape (1, 3, 224, 640)
Figure(1600x800)


# Run YOLO predictions for sample video

In [ ]:
# load model
model = YOLO("runs/detect/train/weights/best.pt")

# run model on sample input video
result = model('results/sample2.mp4', save=True)


WARNING ⚠️ 
Inference results will accumulate in RAM unless `stream=True` is passed, which can cause out-of-memory errors for large
sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/678) /content/drive/MyDrive/kitti_yolo_project/Kitti_Yolo_Tracking/results/sample2.mp4: 384x640 10 Pedestrians, 1 Cyclist, 300.8ms
video 1/1 (frame 2/678) /content/drive/MyDrive/kitti_yolo_project/Kitti_Yolo_Tracking/results/sample2.mp4: 384x640 8 Pedestrians, 1 Cyclist, 151.0ms
video 1/1 (frame 3/678) /content/drive/MyDrive/kitti_yolo_project/Kitti_Yolo_Tracking/results/sample2.mp4: 384x640 1 Car, 7 Pedestrians, 1 Cyclist, 155.5ms
vid

# Trigger Benchmarking
 - **FP32_CPU vs INT8_CPU**
 - **FP32_GPU vs FP16_GPU**

In [ ]:
from benchmark_model import trigger_all_benchmarks
results = trigger_all_benchmarks("runs/detect/train/weights/best.pt", "kitti.yaml")
print(results)


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.

=== FP32 Evaluation ===
Ultralytics 8.4.14 🚀 Python-3.12.12 torch-2.9.0+cu128 CPU (Intel Xeon CPU @ 2.00GHz)
Model summary (fused): 73 layers, 3,006,233 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access ✅ (ping: 0.4±0.1 ms, read: 0.9±0.2 MB/s, size: 789.3 KB)
val: Scanning /content/drive/MyDrive/Kitti_Yolo_Tracking/kitti_yolo/labels/val.cache... 1497 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 1497/1497 285.4Mit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 94/94 2.5s/it 3:57
                   all       1497       6882      0.883      0.784      0.865      0.596
                   Car       133


=== INT8 Evaluation (ONNX Runtime) ===
Ultralytics 8.4.14 🚀 Python-3.12.12 torch-2.9.0+cu128 CPU (Intel Xeon CPU @ 2.00GHz)
Loading runs/detect/train/weights/best.onnx for ONNX Runtime inference...
Using ONNX Runtime 1.24.1 with CPUExecutionProvider
Setting batch=1 input of shape (1, 3, 640, 640)
val: Fast image access ✅ (ping: 0.4±0.1 ms, read: 247.2±61.5 MB/s, size: 826.8 KB)
val: Scanning /content/drive/MyDrive/Kitti_Yolo_Tracking/kitti_yolo/labels/val.cache... 1497 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 1497/1497 448.5Mit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 1497/1497 6.5it/s 3:52
                   all       1497       6882      0.881      0.793      0.868      0.598
                   Car       1333       5680       0.92       0.91       0.96      0.778
            Pedestrian        357        896       0.86       0.68      0.795      0.445
               Cyclist        222        306     

# Generate prediction label files (YOLO format)

In [ ]:
results = model.predict(
    source="data/training/image_2",
    save_txt=True,     # saves YOLO-format predictions
    save_conf=True,    # include confidence scores
    project="runs/detect",
    name="predict",
)


WARNING 
Inference results will accumulate in RAM unless `stream=True` is passed, which can cause out-of-memory errors for large
sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

image 1/7481 H:\GitHub\Kitti_Yolo_Tracking\data\training\image_2\000000.png: 224x640 1 Pedestrian, 162.7ms
image 2/7481 H:\GitHub\Kitti_Yolo_Tracking\data\training\image_2\000001.png: 224x640 1 Car, 1 Cyclist, 90.4ms
image 3/7481 H:\GitHub\Kitti_Yolo_Tracking\data\training\image_2\000002.png: 224x640 1 Car, 89.7ms
image 4/7481 H:\GitHub\Kitti_Yolo_Tracking\data\training\image_2\000003.png: 224x640 2 Cars, 86.3ms
image 5/7481 H:\GitHub\Kitti_Yolo_Tracking\da

# Convert prediction label files: YOLO -> KITTI

In [12]:
!python label_convertor.py "yolo2kitti"

# Generate FAISS Vector index and document for summary store

In [21]:
!python generate_faiss_doc.py 


Loading weights: 100%|##########| 103/103 [00:00<00:00, 1767.64it/s, Materializing param=pooler.dense.weight]
BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.

Generating error documents: 100%|##########| 7481/7481 [00:05<00:00, 1395.46it/s]
